In [1]:
%load_ext autoreload
%autoreload 2

import os
import torch

import sys
sys.path.append("/home/msst/repo/Quantization")
import qlib

model_name='Llama2-7b-hf'
#model_name='Llama-3.2-1B'

DTYPE = torch.float16
model = qlib.load_model(model_name=model_name, torch_dtype=DTYPE)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

### Save fp per block

In [5]:
path_to_save = f'/mnt/ssd_storage/ml/weights/vc_data/{model_name}/per_block_fp'
#os.makedirs(path_to_save, exist_ok=False)


lm_head = model.lm_head
torch.save(lm_head, os.path.join(path_to_save, f'lm_head'))

decoder = model.get_decoder()
embed_tokens = decoder.embed_tokens
torch.save(embed_tokens, os.path.join(path_to_save, f'model.embed_tokens'))

rotary_emb=decoder.rotary_emb
torch.save(rotary_emb, os.path.join(path_to_save, f'model.rotary_emb'))

model_norm=decoder.norm
torch.save(model_norm, os.path.join(path_to_save, f'model.norm'))


for block_id, block in enumerate(decoder.layers):
	torch.save(block, os.path.join(path_to_save, f'model.layers.{block_id}'))

### Save quantized_blocks

In [2]:
#path_to_vc_data = '/mnt/ssd_storage/ml/weights/vc_data/Llama2-7b-hf/kmeans/cb32768_vecdim8_weightPERCOORD_scaleNone_distMSE_blocksizeNone_iters5'
path_to_vc_data = '/mnt/ssd_storage/ml/weights/vc_data/Llama2-7b-hf/kmeans/cb256_vecdim8_weightPERCOORD_scaleOUTL2_distMSE_blocksizeNone_iters25_abscoords'

wrapper = qlib.HomeQuantWrapper(
    wrap_rule={
        torch.nn.Linear : qlib.HQLinear(path_to_vc_data=path_to_vc_data),
    },
    exceptions = {
        'lm_head' : None,
    }
)
wrapper.wrap_model(model)

os.makedirs(f'/home/msst/repo/Quantization/logs/checkpoints_{model_name}', exist_ok=True)
torch.save(model, f'/home/msst/repo/Quantization/logs/checkpoints_{model_name}/kmeans_scaleL2_percoord.pth')

In [9]:
#model.get_decoder().layers[0].self_attn.q_proj.trainable

False

In [3]:
path_to_save = f'/mnt/ssd_storage/ml/weights/vc_data/{model_name}/per_block_q/kmeans_noscale_percoord'
os.makedirs(path_to_save, exist_ok=False)

lm_head = model.lm_head
torch.save(lm_head, os.path.join(path_to_save, f'lm_head'))

decoder = model.get_decoder()
embed_tokens = decoder.embed_tokens
torch.save(embed_tokens, os.path.join(path_to_save, f'model.embed_tokens'))

rotary_emb=decoder.rotary_emb
torch.save(rotary_emb, os.path.join(path_to_save, f'model.rotary_emb'))

for block_id, block in enumerate(decoder.layers):
	torch.save(block, os.path.join(path_to_save, f'model.layers.{block_id}'))

### Save symmetric quant

In [2]:
name = 'cb4096_vecdim8_weightPERCOORD_scaleOUTL2_distMSE_blocksizeNone_iters10_abscoords'
path_to_vc_data = f'/mnt/ssd_storage/ml/weights/vc_data/Llama2-7b-hf/kmeans/{name}'

wrapper = qlib.HomeQuantWrapper(
    wrap_rule={
        torch.nn.Linear : qlib.SymHQLinear(path_to_vc_data=path_to_vc_data),
    },
    exceptions = {
        'lm_head' : None,
        '.0.self_attn' : None,
        '.1.self_attn' : None,
        '.2.self_attn' : None,
    }
)
wrapper.wrap_model(model)

os.makedirs(f'/home/msst/repo/Quantization/logs/checkpoints_{model_name}/SymQuant', exist_ok=True)
torch.save(model, f'/home/msst/repo/Quantization/logs/checkpoints_{model_name}/SymQuant/{name}.pth')

In [3]:
path_to_save = f'/mnt/ssd_storage/ml/weights/vc_data/{model_name}/per_block_q/{name}'
os.makedirs(path_to_save, exist_ok=True)

lm_head = model.lm_head
torch.save(lm_head, os.path.join(path_to_save, f'lm_head'))

decoder = model.get_decoder()
embed_tokens = decoder.embed_tokens
torch.save(embed_tokens, os.path.join(path_to_save, f'model.embed_tokens'))

rotary_emb=decoder.rotary_emb
torch.save(rotary_emb, os.path.join(path_to_save, f'model.rotary_emb'))

for block_id, block in enumerate(decoder.layers):
	torch.save(block, os.path.join(path_to_save, f'model.layers.{block_id}'))

### Save model with new blocks

In [2]:
name = 'cb256_vecdim8_weightPERCOORD_scaleOUTL2_distMSE_blocksizeNone_iters25_abscoords'
path_to_vc_data = f'/mnt/ssd_storage/ml/weights/vc_data/Llama2-7b-hf/kmeans/{name}'

wrapper = qlib.HomeQuantWrapper(
    wrap_rule={
        torch.nn.Linear : qlib.SymHQLinear(path_to_vc_data=path_to_vc_data),
    },
    exceptions = {
        'lm_head' : None,
		'.0.' : None,
		'.1.' : None,
		'.2.' : None,
    }
)
wrapper.wrap_model(model)

In [3]:
path_to_new_blocks = '/home/msst/repo/Quantization/logs/llama2-7b/symquant/llama2_symquant_cb256_vecdim8_C5e-5_L5e-5_lossL1_reassfrac1.0/per_block_q_trained_ns400_bs4_ne2'
#path_to_new_blocks = '/home/msst/repo/Quantization/logs/llama2-7b/symquant/llama2_symquant_cb256_vecdim8_C1e4_lossL2/per_block_q_trained'
block_names = sorted(os.listdir(path_to_new_blocks), key=lambda x: int(x.split('.')[-1]) if x.startswith("model.layers") else -1)


for block_name in block_names:
    new_block = torch.load(os.path.join(path_to_new_blocks, block_name), weights_only=False)
    curr_block = model.get_submodule(block_name)

    with torch.no_grad():
        curr_block.load_state_dict(new_block.state_dict())


In [4]:
torch.save(model, '/home/msst/repo/Quantization/logs/checkpoints_Llama2-7b-hf/SymQuant/cb256_vecdim8_weightPERCOORD_scaleOUTL2_distMSE_blocksizeNone_iters25_abscoords_ptq_3_layers_fp_.pth')